In [1]:
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go


data_path = f'{os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))}/data'
# data_path



In [5]:
def compare(exact, approx, scenario, retval='raw'):
    
    df_exact = pd.read_csv(f'{data_path}/{exact}/NoPerDay_{scenario["NoPerDay"]}_measure_{scenario["measure"]}.csv')
    df_approx = pd.read_csv(f'{data_path}/{approx}/NoPerDay_{scenario["NoPerDay"]}_measure_Avg.csv')
    
    df_merged = df_exact.merge(df_approx, 
                   how='inner', 
                   on=['NoPerDay','RegistrationTime','VaccineTime','WaitingTime','RegistrationDesks','VaccineDesks','SeatingCap'],
                  )
    multiplier = {
        'Avg': 1,
        '90': 2.645,
        '95': 2.960
    }
    df_merged['W1_err'] = (df_merged['W1_x'] - (df_merged['W1_y'] * multiplier[scenario["measure"]]))
    df_merged['W2_err'] = (df_merged['W2_x'] - (df_merged['W2_y'] * multiplier[scenario["measure"]]))
    df_merged['W3_err'] = (df_merged['W3_x'] - (df_merged['W3_y'] * multiplier[scenario["measure"]]))
    df_merged['QueueOutside_err'] = (df_merged['QueueOutside_x'] - (df_merged['QueueOutside_y'] * multiplier[scenario["measure"]]))
    
    df_merged['W1_err_pct'] = (df_merged['W1_x'] - (df_merged['W1_y'] * multiplier[scenario["measure"]]))/df_merged['W1_x']
    df_merged['W2_err_pct'] = (df_merged['W2_x'] - (df_merged['W2_y'] * multiplier[scenario["measure"]]))/df_merged['W2_x']
    df_merged['W3_err_pct'] = (df_merged['W3_x'] - (df_merged['W3_y'] * multiplier[scenario["measure"]]))/df_merged['W3_x']
    df_merged['QueueOutside_err_pct'] = (df_merged['QueueOutside_x'] - (df_merged['QueueOutside_y'] * multiplier[scenario["measure"]]))/df_merged['QueueOutside_x']


    

    if retval== 'stat':
        return df_merged[['W1_err', 'W2_err', 'W3_err', 'QueueOutside_err', 'W1_err_pct', 'W2_err_pct', 'W3_err_pct', 'QueueOutside_err_pct']].describe()
    if retval== 'raw':
        return df_merged[['W1_err', 'W2_err', 'W3_err', 'QueueOutside_err', 'W1_err_pct', 'W2_err_pct', 'W3_err_pct', 'QueueOutside_err_pct']]



In [6]:
approx = 'results_cleaned_20210612'
exact = 'results_cleaned_20210527'

scenario = {
    'NoPerDay': 2200,
    'measure': '95'
}

res = compare(exact, approx, scenario, 'stat')
res
# type(res)

,W1_err,W2_err,W3_err,QueueOutside_err,W1_err_pct,W2_err_pct,W3_err_pct,QueueOutside_err_pct
count,87500.00000,87500.00000,87500.000000,87500.000000,87500.000000,87500.000000,87500.000000,87500.000000
mean,-0.79920,-0.79920,-1.529143,0.912937,-0.434129,-0.434129,-1.401755,0.779577
std,0.99115,0.99115,0.732723,0.187353,0.737717,0.737717,0.882727,0.376432
min,-2.92000,-2.92000,-1.960000,0.278400,-1.960000,-1.960000,-1.960000,0.001569
25%,-1.88000,-1.88000,-1.960000,0.944000,-0.480000,-0.480000,-1.960000,0.496800
50%,-0.84000,-0.84000,-1.960000,1.000000,-0.017773,-0.017773,-1.960000,1.000000
75%,0.04000,0.04000,-1.040000,1.000000,0.000148,0.000148,-0.014857,1.000000
max,1.24000,1.24000,0.160000,1.000000,0.013333,0.013333,0.013333,1.000000


In [ ]:
df = pd.DataFrame(columns=['NoPerDay', 'measure', 'statistic', 'W1_err', 'W2_err', 'W3_err', 'QueueOutside_err'])
approx = 'results_cleaned_20210612'
exact = 'results_cleaned_20210527'

w1_fig = go.Figure()
w2_fig = go.Figure()
w3_fig = go.Figure()
qo_fig = go.Figure()


for NoPerDay in range(1500, 2501, 50):
    for measure in ['Avg', '90', '95']:
        scenario = {
            'NoPerDay': NoPerDay,
            'measure': measure
        }
        res = compare(exact, approx, scenario, 'raw')
#         for stat in ['mean', 'std']:
#             df2 = pd.DataFrame(
#                     [[NoPerDay, measure, stat, res.loc[stat]['W1_err'], res.loc[stat]['W2_err'], res.loc[stat]['W3_err'], res.loc[stat]['QueueOutside_err']]], 
#                     columns=['NoPerDay', 'measure', 'statistic', 'W1_err', 'W2_err', 'W3_err', 'QueueOutside_err'],
#                 )
# #             print(df2)
#             df = df.append(df2, ignore_index=True)
        w1_fig.add_trace(go.Box(y=res['W1_err'], name=f'{NoPerDay}_{measure}'))
        w2_fig.add_trace(go.Box(y=res['W2_err'], name=f'{NoPerDay}_{measure}'))
        w3_fig.add_trace(go.Box(y=res['W3_err'], name=f'{NoPerDay}_{measure}'))
        qo_fig.add_trace(go.Box(y=res['QueueOutside_err'], name=f'{NoPerDay}_{measure}'))


    print(NoPerDay)
#     
#     break
        

# w1_fig.show()
# w2_fig.show()
# w3_fig.show()
# qo_fig.show()

# df

In [ ]:
w1_fig.show()

In [ ]:
w2_fig.show()

In [ ]:
w3_fig.show()

In [ ]:
qo_fig.show()

In [7]:
df = pd.DataFrame(columns=['NoPerDay', 'measure', 'statistic', 'W1_err', 'W2_err', 'W3_err', 'QueueOutside_err', 'W1_err_pct', 'W2_err_pct', 'W3_err_pct', 'QueueOutside_err_pct'])
approx = 'results_cleaned_20210612'
exact = 'results_cleaned_20210527'

w1_fig = go.Figure()
w2_fig = go.Figure()
w3_fig = go.Figure()
qo_fig = go.Figure()


for NoPerDay in range(1500, 2501, 50):
    for measure in ['Avg', '90', '95']:
        scenario = {
            'NoPerDay': NoPerDay,
            'measure': measure
        }
        res = compare(exact, approx, scenario, 'stat')
        for stat in ['mean', 'std']:
            df2 = pd.DataFrame(
                    [[NoPerDay, measure, stat, res.loc[stat]['W1_err'], res.loc[stat]['W2_err'], res.loc[stat]['W3_err'], res.loc[stat]['QueueOutside_err'], res.loc[stat]['W1_err_pct'], res.loc[stat]['W2_err_pct'], res.loc[stat]['W3_err_pct'], res.loc[stat]['QueueOutside_err_pct']]], 
                    columns=['NoPerDay', 'measure', 'statistic', 'W1_err', 'W2_err', 'W3_err', 'QueueOutside_err', 'W1_err_pct', 'W2_err_pct', 'W3_err_pct', 'QueueOutside_err_pct'],
                )
            df = df.append(df2, ignore_index=True)

    print(NoPerDay)
#     
#     break

print(df)

1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
    NoPerDay measure statistic    W1_err    W2_err    W3_err  \
0       1500     Avg      mean  0.123077  0.126984  0.000000   
1       1500     Avg       std  0.328526  0.332956  0.000000   
2       1500      90      mean -0.995231 -0.999286 -1.488171   
3       1500      90       std  0.956322  0.970470  0.424077   
4       1500      95      mean -1.232615 -1.235556 -1.749268   
..       ...     ...       ...       ...       ...       ...   
121     2500     Avg       std  0.287482  0.287482  0.173994   
122     2500      90      mean -0.642841 -0.642841 -1.185937   
123     2500      90       std  0.911922  0.911922  0.778820   
124     2500      95      mean -0.833636 -0.833636 -1.393750   
125     2500      95       std  1.021485  1.021485  0.916310   

     QueueOutside_err  W1_err_pct  W2_err_pct  W3_err_pct  \
0            0.954390    0.005995    0.006185    0.000000   
1   

In [8]:
df.to_csv('approximation_error.csv')

In [ ]:
queue_outside_df = df[(df['measure']=='Avg') & (df['statistic']=='mean')]

px.line(queue_outside_df, x='NoPerDay', y='W2_err')

In [10]:
[3] + list((4, 5))

[3, 4, 5]